In [ ]:
#imports
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt

import re, ast
from scipy.sparse import csr_matrix, hstack, vstack, issparse

from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [ ]:
products_df = pd.read_csv('../data/amazon_meta.csv')
reviews_df = pd.read_csv('../data/amazon_reviews.csv')

In [ ]:
df = pd.merge(products_df, reviews_df, on='parent_asin', how='inner')

In [ ]:
item_df = (
    df.groupby('parent_asin')
    .agg(
        price = ('price', 'mean'),
        rating = ('rating', 'mean'),
        num_ratings = ('rating', 'count'),
        text = ('merged_text', 'first'),
    )
)

In [ ]:
print(products_df.dtypes)
print(reviews_df.dtypes)

In [ ]:
# Creating combined text feature for categorical/text data
item_df['combined_text'] = (
    df['product_title'].apply(lambda x: " ".join(x)) + " " +
    df['reviews_title'].apply(lambda x: " ".join(x)) + " " +
    df['text'].apply(lambda x: " ".join(x)) + " " +
    df['categories'].apply(lambda x: " ".join(x)) + " "
)

In [ ]:
# tf-idf transformation
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_matrix = tfidf.fit_transform(item_df['combined_text'])

In [ ]:
# standard scaling for numeric features
numeric_features = ['price']
scaler = StandardScaler()

numeric_scaled = scaler.fit_transform(item_df[numeric_features])

In [ ]:
# Creating matrices
numeric_sparse = csr_matrix(np.nan_to_num(numeric_scaled))
hybrid_matrix = hstack([tfidf_matrix, numeric_sparse])

tfidf_index = df[['parent_asin', 'title']].reset_index(drop=True)